In [ ]:
! pip install pandas
! pip install numpy
! pip install matplotlib
! pip install seaborn
! pip install scikit-learn


# 0 - Imports

In [3]:
import pandas as pd
pd.set_option("display.max_colwidth", 150)
pd.set_option("display.min_rows", 20)

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.cluster import KMeans, DBSCAN
from sklearn.decomposition import PCA
from sklearn.preprocessing import LabelEncoder, StandardScaler
from datetime import datetime
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report

# 1 - Carregar o dataset de treino

In [31]:
df = pd.read_csv("./csv/treino/clientes.csv")

In [ ]:
df.head()

# 2 - Pré-processamento dos dados

In [ ]:
label_encoders = {}
for column in ['Sexo', 'Profissão', 'Estado Civil']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Preenchendo valores ausentes
df.fillna({'Data de Início no Último Emprego': pd.to_datetime('2000-01-01')}, inplace=True)
df['Data de Início no Último Emprego'] = pd.to_datetime(df['Data de Início no Último Emprego'])
df['Anos de Emprego'] = (pd.to_datetime('today') - df['Data de Início no Último Emprego']).dt.days // 365
df


# 3 - Adicionar idade e risco baseado em condições

In [ ]:
df['Data de Nascimento'] = pd.to_datetime(df['Data de Nascimento'])
df["Idade"] = (datetime.now() - df['Data de Nascimento']).dt.days // 365

# Definindo o risco com base na idade e outras condições
def calculate_risk(row):
    risk = 0
    # Condições de risco baseados na idade
    if 18 <= row["Idade"] <= 24:
        risk += 1  # Risco médio
    elif 25 <= row['Idade'] <= 49:
        risk -= 1 # Risco baixo
    else:
        risk += 1 # Risco médio

    # Adicionando risco se a Data de Início do Último Emprego for nulo
    if row['Data de Início no Último Emprego'] == pd.to_datetime('2000-01-01'):
        risk += 4  # Aumenta o risco em 80%

    # Adicionando risco baseado no sexo
    if row['Sexo'] == 1:
        risk += 0.05  # 

    return risk 

df['Risco'] = df.apply(calculate_risk, axis=1)

# Somar o risco ao score do crédito
df['Score Ajustado'] = 500 + (df['Risco'] * 50) # Inicializa com um score base de 500
df['Score Ajustado'] = df['Score Ajustado'].clip(lower=0, upper=850)

df

# 4 - Transformando o Score Ajustado em Classes

In [ ]:
def categorize_score(score):
    if score >= 600:
        return 1  # Bom pagador
    elif 400 <= score < 600:
        return 0  # Pagador médio
    else:
        return -1  # Mal pagador
    

# Aplicando a categorização
df['Classificacao'] = df['Score Ajustado'].apply(categorize_score)

# Separar em variáveis de entrada e variáveis alvo
X = df.drop(columns=['Score Ajustado', 'Classificacao', 'Nome', 'Data de Nascimento', 'Data de Início no Último Emprego'])
y = df['Classificacao']

df

In [36]:
# Normalizando os dados
scaler = StandardScaler()
x_scaled = scaler.fit_transform(X)

# Dividindo o dataset em treino e teste
x_train, x_test, y_train, y_test = train_test_split(x_scaled, y, test_size=0.3, random_state=42)


5. Treinamento e Teste com KMN

In [37]:
knm = KNeighborsClassifier(n_neighbors=5)
knm.fit(x_train, y_train)
y_pred_knm = knm.predict(x_test)


# 6. Treinamento e Teste com Árvore de Decisão

In [38]:
tree = DecisionTreeClassifier(random_state=42)
tree.fit(x_train, y_train)
y_pred_tree = tree.predict(x_test)


# 7. Comparando os Modelos

In [ ]:
print("Resultado do KMN:")
print(classification_report(y_test, y_pred_knm))

print('Resultado da Árvore de Decisão:')
print(classification_report(y_test, y_pred_tree))

# Visualizações (Boxplot, K-Means e DBSCAN)

# 8. Boxplot para detectar outliers

In [ ]:
plt.Figure(figsize=(10,6))
sns.boxplot(data=df[['Salário Atual', 'Anos de Emprego']])
plt.title('Detecção de Outliers - Salário Base e Anos no Emprego')
plt.show()

# 9. K-Means para visualizar clusters de clientes

In [ ]:
pca = PCA(n_components=2)
X_pca = pca.fit_transform(x_scaled)

# Clustering com K-Means
kmeans = KMeans(n_clusters=3, random_state=42)
clusters = kmeans.fit_predict(X_pca)

plt.Figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=clusters, cmap='viridis')
plt.title('Clusters de Clientes - K-Means')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()


# 10. DBSCAN para visualização de quadrantes

In [ ]:
dbscan = DBSCAN(eps=0.5, min_samples=5)
db_clusters = dbscan.fit_predict(X_pca)

plt.figure(figsize=(10, 6))
plt.scatter(X_pca[:, 0], X_pca[:, 1], c=db_clusters, cmap='plasma')
plt.title('DBSCAN - Clusters em Quadrantes')
plt.xlabel('Componente Principal 1')
plt.ylabel('Componente Principal 2')
plt.show()

# Testar um novo dataset

In [46]:
df_teste = pd.read_csv('./csv/teste/clientes_testes.csv')

In [ ]:
df_teste.head()

# Pré-processamento do novo dataset

In [ ]:
df_teste['Data de Nascimento'] = pd.to_datetime(df_teste['Data de Nascimento'])
df_teste['Data de Início no Último Emprego'] = pd.to_datetime(df_teste['Data de Início no Último Emprego'], errors='coerce')
df_teste.fillna({'Data de Início no Último Emprego': pd.to_datetime('2000-01-01')}, inplace=True)
df_teste['Idade'] = (datetime.now() - df_teste['Data de Nascimento']).dt.days // 365
df_teste['Anos no Emprego'] = (pd.to_datetime('today') - df_teste['Data de Início no Último Emprego']).dt.days // 365
df_teste['Risco'] = df_teste.apply(calculate_risk, axis=1)
df_teste['Score Ajustado'] = df_teste['Score de Crédito'] + (df_teste['Risco'] * 50)
df_teste['Score Ajustado'] = df_teste['Score Ajustado'].clip(lower=0, upper=850)
df_teste['Classificacao'] = df_teste['Score Ajustado'].apply(categorize_score)

# Preprocessando os dados do novo dataset

In [52]:
x_teste = df_teste[['Sexo', 'Profissão', 'Salário Atual', 'Estado Civil', 'Número de Filhos', 'Empregado', 'Idade', 'Anos no Emprego']]
x_teste = pd.get_dummies(x_teste, drop_first=True) # One-hot encoding para variáveis categóricas

In [53]:
# Garantindo a mesma ordem das colunas do dataset de treino
x_teste = x_teste.reindex(columns=X.columns, fill_value=0) # Preencher com 0 para colunas que não estão no teste
x_teste_scaled = scaler.transform(x_teste)  # Normalizando os dados do novo dataset


In [54]:
# Realizando previsões no novo dataset
predicoes_kmn = knm.predict(x_teste_scaled)
predicoes_tree = tree.predict(x_teste_scaled)

In [55]:
# Adicionando as previsões ao dataset
df_teste['Predição KMN'] = predicoes_kmn
df_teste['Predição Arvore'] = predicoes_tree

In [ ]:
df_teste

In [57]:
# Função para mapear as previsões para labels descritivos
def map_prediction(pred):
    if pred == 1:
        return 'Bom pagador'
    elif pred == 0:
        return 'Pagador médio'
    else:
        return 'Mal pagador'
    

In [58]:
# Imprimindo os resultados das previsões com labels descritivos
df_teste['Predição KMN'] = df_teste['Predição KMN'].apply(map_prediction)
df_teste['Predição Arvore'] = df_teste['Predição Arvore'].apply(map_prediction)

print('\nResultado das Previsões:')
print(df_teste[['Nome', 'Predição KMN', 'Predição Arvore']])


Resultado das Previsões:
            Nome   Predição KMN Predição Arvore
0      Cliente_0  Pagador médio     Bom pagador
1      Cliente_1  Pagador médio     Bom pagador
2      Cliente_2  Pagador médio     Bom pagador
3      Cliente_3  Pagador médio   Pagador médio
4      Cliente_4  Pagador médio     Bom pagador
5      Cliente_5  Pagador médio     Bom pagador
6      Cliente_6  Pagador médio     Bom pagador
7      Cliente_7  Pagador médio   Pagador médio
8      Cliente_8  Pagador médio   Pagador médio
9      Cliente_9  Pagador médio   Pagador médio
..           ...            ...             ...
990  Cliente_990  Pagador médio   Pagador médio
991  Cliente_991  Pagador médio   Pagador médio
992  Cliente_992  Pagador médio   Pagador médio
993  Cliente_993  Pagador médio   Pagador médio
994  Cliente_994  Pagador médio     Bom pagador
995  Cliente_995  Pagador médio   Pagador médio
996  Cliente_996  Pagador médio   Pagador médio
997  Cliente_997  Pagador médio     Bom pagador
998  Cliente_9

# Exportando o resultado para um CSV

In [59]:
df_teste[['Nome', 'Predição KMN', 'Predição Arvore']].to_csv('resultados_previsao.csv', index=False)
